# BYOA の R スクリプトに対してハイパーパラメータのチューニングを行う

#### ノートブックに含まれる内容

- Bring Your Own Algorithm(BYOA) におけるハイパーパラーメータチューニングの方法の説明
- ECR によるコンテナの利用法（**<span style="color: red;">このノートブックを実行するには，通常の `SageMakerFullAccess` に加えて `AmazonEC2ContainerRegistryFullAccess` が適用されているロールを使用する必要があります</span>**）

#### ノートブックで使われている手法の詳細

- アルゴリズム: MARS(Multivariate Adaptive Regression Splines)
- データ: iris

## セットアップ

必要なパラメタをセットアップします．


In [ ]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()

以下を実行する前に，**<span style="color: red;">`sagemaker/hpo-r-byoa/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/hpo-r-byoa/XX'

手元の `Dockerfile` をビルドして，ECR に `push` します．

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=rmars

#set -e # stop if anything fails
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## データのロード

手元の `iris.csv` をS3 にアップロードします．

In [ ]:
train_file = 'iris.csv'
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_file(train_file)

## ハイパーパラメータチューニングジョブの実行

続いて，チューニングジョブのセットアップを行い実行します．ここには以下の 4 つの処理が含まれます．

1. 通常の学習ジョブのときと同様に，Tensorflow クラスのオブジェクトを作成します
1. チューニングしたいハイパーパラメータ名と範囲を，Dictionary 型で指定します
1. チューニングの評価を行うためのターゲットメトリクスを指定します
1. 実際にチューニングジョブを実行します

### 1. Estimater オブジェクトの作成

これは，通常の学習ジョブを実行するとの全く同じ手順です．ここでは ECR に `push` したコンテナ ID を取得して，それを指定した形で Estimater オブジェクトを作成します．

In [ ]:
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_name='{}.dkr.ecr.{}.amazonaws.com/rmars:latest'.format(account, region),
    role=role,
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker.Session(),
    hyperparameters={'target': 'Sepal.Length'})

### 2. ハイパーパラメータのリストを作成

次に，チューニングしたいハイパーパラメータのリストを作成します．ハイパーパラメータの中身に応じたオブジェクトがあるので，これを使用します．ハイパーパラメータがカテゴリの場合は探索対象のカテゴリのリストを，連続値の場合は範囲を指定する形にしてください．なお整数の場合は，通常の連続値とは異なるオブジェクトを用いて指定します．

- カテゴリ: `CategoricalParameter(list)`
- 連続値: `ContinuousParameter(min, max)`
- 整数: `IntegerParameter(min, max)`

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'degree': IntegerParameter(1, 3),
                         'thresh': ContinuousParameter(0.001, 0.01),
                         'prune': CategoricalParameter(['TRUE', 'FALSE'])}

### 3. ターゲットメトリクスの指定

続いて，チューニングの評価をするためのメトリクスを指定します．このメトリクスは，Sagemaker 側でジョブ実行時の標準出力から正規表現で抽出します．対象となるメトリクスが標準出力ログに出力されるように，`mars.R` の中で `print` メソッドを記述します．ここでは，mse がターゲットとメトリクスとなりますので，それを出力した上で，ログから抽出するための正規表現を以下に記述します．

In [ ]:
objective_metric_name = 'mse'
metric_definitions = [{'Name': 'mse',
                       'Regex': 'mse: ([0-9\\.]+)'}]

### 4. チューニングジョブの実行

以上の準備が終わったら，チューニングジョブのオプビェクトを作成して，`fit()` で実際に実行します．その際に **<span style="color: red;">`base_tuning_job_name` の `hpo-r-byoa-XX` にある `XX` を指定された適切な数字に変更</span>**してください

`HyperparameterTuner` の詳細については[ドキュメント](https://sagemaker.readthedocs.io/en/latest/tuner.html)をご確認ください．


In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            objective_type='Minimize',
                            max_jobs=9,
                            max_parallel_jobs=3,
                            base_tuning_job_name='hpo-r-byoa-XX')

tuner.fit({'train': 's3://{}/{}/train'.format(bucket, prefix)})

チューニングジョブの実行状況は，`boto3` クライアント経由で確認することが可能です．

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']